In [78]:
import cv2
import os
import csv
import numpy as np
import mediapipe as mp
from matplotlib import pyplot as plt
import time

In [79]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [80]:
def mediapipe_detection(frame, model):
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    return image, results

In [81]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
   # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [82]:
def record_landmarks(results, action):
    try:
        '''
        pose = results.pose_landmarks.landmark
        pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

        right_hand = results.right_hand_landmarks.landmark
        right_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in right_hand]).flatten())

        left_hand = results.left_hand_landmarks.landmark
        left_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in left_hand]).flatten())
        '''

        if results.pose_landmarks:
            pose_row = list(np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten())
        else:
            pose_row = list(np.zeros(132).flatten())
        
        if results.left_hand_landmarks:
            lh_row = list(np.array([[res.x, res.y, res.z, res.visibility] for res in results.left_hand_landmarks.landmark]).flatten())
        else:
            lh_row = list(np.zeros(84).flatten())
        
        if results.right_hand_landmarks:
            rh_row = list(np.array([[res.x, res.y, res.z, res.visibility] for res in results.right_hand_landmarks.landmark]).flatten())
        else:
            rh_row = list(np.zeros(84).flatten())
        
        row = pose_row + right_hand_row + left_hand_row
        row.insert(0, action)

        with open('coords6.csv', mode='a', newline='') as f:
            csv_writer_obj = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer_obj.writerow(row)
            
    except:
        pass





        

In [83]:
len(results.pose_landmarks.landmark)*4

132

# Testing Camera and recording

In [84]:
#TESTING CAMERA

cam = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.8, min_tracking_confidence=0.5) as holistic:
    while cam.isOpened():
        ret, frame = cam.read()
    
        image, results = mediapipe_detection(frame, holistic)
        
        draw_landmarks(image, results)
        
        cv2.imshow("Sign Language Converter", cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        #print(results.right_hand_landmarks) 
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cam.release()
cv2.destroyAllWindows()

In [85]:
cam.release()
cv2.destroyAllWindows()

In [95]:
#CREATE HEADER IN CSV
NUM_COORDS = 75
#num_coords = len(results.pose_landmarks.landmark) + len(results.right_hand_landmarks.landmark) + len(results.left_hand_landmarks.landmark)
header_list = ["action"]
for i in range(1, NUM_COORDS+1):
    header_list += ["x{}".format(i),"y{}".format(i),"z{}".format(i),"v{}".format(i)]

with open('coords6.csv', mode='w', newline='') as f:
    csv_writer_obj = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer_obj.writerow(header_list)

In [96]:
DATA_PATH = os.path.join('MP_data')
actions = np.array(["Police","Danger","Help"])
no_sequences = 10
sequence_length = 30

In [97]:
#recording per sequence

cam = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.8, min_tracking_confidence=0.5) as holistic:
    
    for action in actions:
        for sequence in range(no_sequences):
            for frame_no in range(sequence_length):
                
                ret, frame = cam.read()
                image, results = mediapipe_detection(frame, holistic)
                draw_landmarks(image, results)
                
                #print(results.right_hand_landmarks)
                #extract_keypoints(results)
                if frame_no == 0:
                    cv2.putText(image, "Collecting Video for {} Video number {}".format(action, sequence), (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 1, cv2.LINE_AA)
                    cv2.waitKey(2000)
                else:
                     cv2.putText(image, "Collecting Video for {} Video number {}".format(action, sequence), (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 1, cv2.LINE_AA)

                record_landmarks(results, action)
                #npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_no))
                #np.save(npy_path, keypoints)
                
                cv2.imshow("Input Sign Language", cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
                
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    cam.release()
                    cv2.destroyAllWindows()

    cam.release()
    cv2.destroyAllWindows()

In [37]:
#RECORDING WITHOUT ANY SEQUENCE

cam = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.8, min_tracking_confidence=0.5) as holistic:
    while cam.isOpened():
        ret, frame = cam.read()
    
        image, results = mediapipe_detection(frame, holistic)
        
        draw_landmarks(image, results)

        record_landmarks(results)
        
        cv2.imshow("Sign Language Converter", cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        #print(results.right_hand_landmarks) 
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cam.release()
cv2.destroyAllWindows()

TypeError: record_landmarks() missing 1 required positional argument: 'action'

# SPLIT DATA

In [98]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [99]:
df = pd.read_csv("coords6.csv")

In [100]:
X = df.drop('action', axis=1) #feature
y = df['action'] #target 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

ValueError: With n_samples=0, test_size=0.3 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [92]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.metrics import accuracy_score
import pickle


In [93]:
#TRAINING

pipelines = {
    'lr' : make_pipeline(StandardScaler(), LogisticRegression()),
    'rc' : make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf' : make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb' : make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [94]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

NameError: name 'X_train' is not defined

In [68]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

In [69]:
fit_models['rf'].predict(X_test)

KeyError: 'rf'

In [70]:
y_test

NameError: name 'y_test' is not defined

In [71]:
with open('body_language.pkl', 'wb') as f:
    pickle.dump(fit_models['lr'], f)

KeyError: 'lr'

In [127]:
with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)

In [128]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

# Real Time Testing

In [130]:
cam = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.8, min_tracking_confidence=0.5) as holistic:
    while cam.isOpened():
        ret, frame = cam.read()
    
        image, results = mediapipe_detection(frame, holistic)
        draw_landmarks(image, results)
        
        #print(results.right_hand_landmarks)

        try:
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
    
            right_hand = results.right_hand_landmarks.landmark
            right_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in right_hand]).flatten())
    
            left_hand = results.left_hand_landmarks.landmark
            left_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in left_hand]).flatten())
     
            row = pose_row + right_hand_row + left_hand_row
            
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)

            '''   coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)'''
            
            # Get status box
            cv2.rectangle(image, (0,0), (640, 35), (0,0,0, 0.5), -1)

            cv2.putText(image, 'Action:'
                        , (10,25), cv2.FONT_HERSHEY_DUPLEX, 0.85, (255, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (110,25), cv2.FONT_HERSHEY_DUPLEX, 0.85, (255, 255, 255), 1, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'Accuracy:'
                        , (420,25), cv2.FONT_HERSHEY_DUPLEX, 0.85, (255, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (560,25), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255, 255, 255), 1, cv2.LINE_AA)
        except:
            pass


        cv2.imshow("Sign Language Converter", cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cam.release()
cv2.destroyAllWindows()


C:\Users\manja\.conda\envs\mp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\manja\.conda\envs\mp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


Help [3.73367327e-10 9.38577596e-01 6.14224038e-02]


C:\Users\manja\.conda\envs\mp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\manja\.conda\envs\mp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\manja\.conda\envs\mp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\manja\.conda\envs\mp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\manja\.conda\envs\mp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted

Help [5.38276149e-06 8.87624006e-01 1.12370611e-01]
Help [2.23676400e-06 9.99997763e-01 6.96841972e-19]
Help [2.33367777e-06 9.99997666e-01 1.91060527e-30]
Help [4.17057485e-09 9.99999996e-01 4.00572144e-37]


C:\Users\manja\.conda\envs\mp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\manja\.conda\envs\mp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


...

Help [8.87359687e-04 9.99071486e-01 4.11540581e-05]
Help [1.23746932e-02 9.87625251e-01 5.54402650e-08]
Help [6.47137250e-03 9.93528288e-01 3.39369657e-07]
Help [1.14742773e-03 9.98850187e-01 2.38547367e-06]


C:\Users\manja\.conda\envs\mp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\manja\.conda\envs\mp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\manja\.conda\envs\mp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\manja\.conda\envs\mp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\manja\.conda\envs\mp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted

Help [4.80738284e-04 9.99518396e-01 8.65579392e-07]
Help [7.60891824e-04 9.99238565e-01 5.43576314e-07]
Help [1.10083935e-03 9.98898672e-01 4.89078471e-07]
Help [3.66447976e-03 9.96334484e-01 1.03589579e-06]


C:\Users\manja\.conda\envs\mp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\manja\.conda\envs\mp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\manja\.conda\envs\mp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\manja\.conda\envs\mp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\manja\.conda\envs\mp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted

Help [9.69145967e-03 9.90307542e-01 9.98804255e-07]
Help [3.37368617e-02 9.66263061e-01 7.70541534e-08]
Help [4.31433561e-03 9.95685664e-01 7.98397795e-10]
